In [14]:
import numpy as np
import time
import pandas as pd
import sys
from Strategy_Evaluation import Strategy_Evaluation

class Market4(object):
    def __init__(self,begin_date="2016/1/1",end_date="2017/5/5"):
        self.action_space = [-1,0,1]
        #action record price,time,action
        self.lstm_length=10
        self.get_price=self.price_data(begin_date,end_date)
        self.action_record=pd.DataFrame(columns=["Date","Vol","ret","price","action"])
        self.ot=list(self.get_price.__next__())
        self.count_episode=0
        self.n_actions=3
        self.acc_value=[]
        self.acc_position=0
        
    
    def price_data(self,begin_date,end_date):
        series=pd.read_csv('price-2.csv')
#         print(series)
        index_list=series.columns
        for i in index_list:
            for x in range(self.lstm_length):
                series[i+"lag"+str(x+1)]=series[i].shift(x+1)
        series=series[(series.Date>begin_date)&(series.Date<end_date)]
        series=series.dropna()
#         print(series)
        self.n_features=series.shape[1]-3
        for index,row in series.iterrows():
            b=row
            xx=pd.concat([b.select(lambda col: col.startswith(i)).reset_index(drop=True)  for i in index_list],axis=1)
            xx.columns=index_list
            yield xx.sort_values("Date",ascending=True)
    

    def step(self, action):
        # record St,Ot,At
        self.ot=self.get_price.__next__()
        self.acc_position+=action
        self.action_record.loc[self.count_episode]=list(self.ot.iloc[0,:])+[action]
        self.evaluation=Strategy_Evaluation(self,initial_account_value=200000)
        # reward function
        self.evaluation.cal_win_rate()
        print(self.evaluation.Win_rate)
        s_ = self.ot
        reward=sum((s_.ret[0]-self.action_record[self.action_record.action != 0].ret)*self.action_record[self.action_record.action != 0].action)
        self.acc_value+=[sum((s_.price[0]-self.action_record[self.action_record.action != 0].price)*self.action_record[self.action_record.action != 0].action)]
        self.count_episode+=1
        return s_,reward

In [15]:
env=Market4()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement


In [16]:
get=env.price_data(begin_date="2016/1/1",end_date="2017/5/5")


In [19]:
np.array(get.__next__().drop(columns=["Date"]))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement


array([[0.0, 57418.0, 9261.0],
       [-0.00541361218203384, 116186.0, 9211.0],
       [-0.0104770138754731, 108371.0, 9115.0],
       [-0.000878059544943355, 69826.0, 9107.0],
       [0.0017553487776549301, 47613.0, 9123.0],
       [-0.0014259860914709302, 97142.0, 9110.0],
       [0.0085255735549854, 92394.0, 9188.0],
       [0.00293430629493585, 116363.0, 9215.0],
       [-0.0168017222301629, 89236.0, 9266.0],
       [0.00161751261880028, 87205.0, 9281.0],
       [-0.0011859200354926698, 118916, 9270]], dtype=object)

In [7]:
"""
This part of code is the DQN brain, which is a brain of the agent.
All decisions are made in here.
Using Tensorflow to build the neural network.

View more on my tutorial page: https://morvanzhou.github.io/tutorials/

Using:
Tensorflow: 1.0
gym: 0.7.3
"""

import numpy as np
import pandas as pd
import tensorflow as tf

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = "0"
# np.random.seed(1)
# tf.set_random_seed(1)


# Deep Q Network off-policy
class DeepQNetwork1:
    def __init__(
            self,
            n_actions,
            n_features,
            learning_rate=0.01,
            reward_decay=0.9,
            e_greedy=0.9,
            replace_target_iter=300,
            memory_size=500,
            batch_size=32,
            e_greedy_increment=None,
            output_graph=False,
        max_position=5,
    ):
        self.n_actions = n_actions
        self.n_features = n_features
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon_max = e_greedy
        self.replace_target_iter = replace_target_iter
        self.memory_size = memory_size
        self.batch_size = batch_size
        self.epsilon_increment = e_greedy_increment
        self.epsilon = 0 if e_greedy_increment is not None else self.epsilon_max
        self.cum_position=0
        # total learning step
        self.learn_step_counter = 0
        self.max_position=max_position
        # initialize zero memory [s, a, r, s_]
        self.memory = np.zeros((self.memory_size, n_features * 2 + 2))

        # consist of [target_net, evaluate_net]
        self._build_net()
        t_params = tf.get_collection('target_net_params')
        e_params = tf.get_collection('eval_net_params')
        self.replace_target_op = [tf.assign(t, e) for t, e in zip(t_params, e_params)]

        self.sess = tf.Session()
        if output_graph:
            # $ tensorboard --logdir=logs
            # tf.train.SummaryWriter soon be deprecated, use following
            tf.summary.FileWriter("logs/", self.sess.graph)

        self.sess.run(tf.global_variables_initializer())
        self.cost_his = []

    def _build_net(self):
        # ------------------ build evaluate_net ------------------
        self.s = tf.placeholder(tf.float32, [None, self.n_features], name='s')  # input
        self.q_target = tf.placeholder(tf.float32, [None, self.n_actions], name='Q_target')  # for calculating loss
        with tf.variable_scope('eval_net'):
            # c_names(collections_names) are the collections to store variables
            c_names, n_l1, w_initializer, b_initializer = \
                ['eval_net_params', tf.GraphKeys.GLOBAL_VARIABLES], 10, \
                tf.random_normal_initializer(0., 0.3), tf.constant_initializer(0.1)  # config of layers

            # first layer. collections is used later when assign to target net
            with tf.variable_scope('l1'):
                w1 = tf.get_variable('w1', [self.n_features, n_l1], initializer=w_initializer, collections=c_names)
                b1 = tf.get_variable('b1', [1, n_l1], initializer=b_initializer, collections=c_names)
                l1 = tf.nn.relu(tf.matmul(self.s, w1) + b1)

            # second layer. collections is used later when assign to target net
            with tf.variable_scope('l2'):
                w2 = tf.get_variable('w2', [n_l1, self.n_actions], initializer=w_initializer, collections=c_names)
                b2 = tf.get_variable('b2', [1, self.n_actions], initializer=b_initializer, collections=c_names)
                self.q_eval = tf.matmul(l1, w2) + b2

        with tf.variable_scope('loss'):
            self.loss = tf.reduce_mean(tf.squared_difference(self.q_target, self.q_eval))
        with tf.variable_scope('train'):
            self._train_op = tf.train.RMSPropOptimizer(self.lr).minimize(self.loss)

        # ------------------ build target_net ------------------
        self.s_ = tf.placeholder(tf.float32, [None, self.n_features], name='s_')    # input
        with tf.variable_scope('target_net'):
            # c_names(collections_names) are the collections to store variables
            c_names = ['target_net_params', tf.GraphKeys.GLOBAL_VARIABLES]

            # first layer. collections is used later when assign to target net
            with tf.variable_scope('l1'):
                w1 = tf.get_variable('w1', [self.n_features, n_l1], initializer=w_initializer, collections=c_names)
                b1 = tf.get_variable('b1', [1, n_l1], initializer=b_initializer, collections=c_names)
                l1 = tf.nn.relu(tf.matmul(self.s_, w1) + b1)

            # second layer. collections is used later when assign to target net
            with tf.variable_scope('l2'):
                w2 = tf.get_variable('w2', [n_l1, self.n_actions], initializer=w_initializer, collections=c_names)
                b2 = tf.get_variable('b2', [1, self.n_actions], initializer=b_initializer, collections=c_names)
                self.q_next = tf.matmul(l1, w2) + b2

    def store_transition(self, s, a, r, s_):
        if not hasattr(self, 'memory_counter'):
            self.memory_counter = 0

        transition = np.hstack((s, [a, r], s_))

        # replace the old memory with new memory
        index = self.memory_counter % self.memory_size
        self.memory[index, :] = transition

        self.memory_counter += 1

    def choose_action(self, observation):
        # to have batch dimension when feed into tf placeholder
        observation = np.array([observation])
        print(observation)
        actions_value = self.sess.run(self.q_eval, feed_dict={self.s: observation})
        if abs(self.cum_position)<self.max_position:
            if np.random.uniform() < self.epsilon:
                # forward feed the observation and get q value for every actions
                action = np.argmax(actions_value)-1
            else:
                action = np.random.randint(0, self.n_actions)-1
        elif abs(self.cum_position)==self.max_position:
            if self.cum_position>0:
                action = np.argmax(actions_value[0][0:2])-1
            else:
                action = np.argmax(actions_value[0][1:3])
        self.cum_position+=action
        return action

    def learn(self):
        # check to replace target parameters
        if self.learn_step_counter % self.replace_target_iter == 0:
            self.sess.run(self.replace_target_op)
            print('\ntarget_params_replaced\n')

        # sample batch memory from all memory
        if self.memory_counter > self.memory_size:
            sample_index = np.random.choice(self.memory_size, size=self.batch_size)
        else:
            sample_index = np.random.choice(self.memory_counter, size=self.batch_size)
        batch_memory = self.memory[sample_index, :]

        q_next, q_eval = self.sess.run(
            [self.q_next, self.q_eval],
            feed_dict={
                self.s_: batch_memory[:, -self.n_features:],  # fixed params
                self.s: batch_memory[:, :self.n_features],  # newest params
            })

        # change q_target w.r.t q_eval's action
        q_target = q_eval.copy()

        batch_index = np.arange(self.batch_size, dtype=np.int32)
        eval_act_index = batch_memory[:, self.n_features].astype(int)
        reward = batch_memory[:, self.n_features + 1]

        q_target[batch_index, eval_act_index] = reward + self.gamma * np.max(q_next, axis=1)

        """
        For example in this batch I have 2 samples and 3 actions:
        q_eval =
        [[1, 2, 3],
         [4, 5, 6]]

        q_target = q_eval =
        [[1, 2, 3],
         [4, 5, 6]]

        Then change q_target with the real q_target value w.r.t the q_eval's action.
        For example in:
            sample 0, I took action 0, and the max q_target value is -1;
            sample 1, I took action 2, and the max q_target value is -2:
        q_target =
        [[-1, 2, 3],
         [4, 5, -2]]

        So the (q_target - q_eval) becomes:
        [[(-1)-(1), 0, 0],
         [0, 0, (-2)-(6)]]

        We then backpropagate this error w.r.t the corresponding action to network,
        leave other action as error=0 cause we didn't choose it.
        """

        # train eval network
        _, self.cost = self.sess.run([self._train_op, self.loss],
                                     feed_dict={self.s: batch_memory[:, :self.n_features],
                                                self.q_target: q_target})
        self.cost_his.append(self.cost)

        # increasing epsilon
        self.epsilon = self.epsilon + self.epsilon_increment if self.epsilon < self.epsilon_max else self.epsilon_max
        self.learn_step_counter += 1

    def plot_cost(self):
        import matplotlib.pyplot as plt
        plt.switch_backend('agg')
        plt.plot(np.arange(len(self.cost_his)), self.cost_his)
        plt.ylabel('Cost')
        plt.xlabel('training steps')
        plt.savefig("image.png")
        plt.close()